# Normalization

## Data Normalization

## Deep Learning Layers Normalization

### Local Reponse Normalization

La **Normalización de Respuesta Local (LRN, por sus siglas en inglés)** es una técnica introducida en las primeras arquitecturas de **redes neuronales convolucionales (CNNs)**, destacando especialmente en **AlexNet (2012)**. Su propósito principal es **mejorar la capacidad de generalización** del modelo y promover la **competencia entre neuronas** dentro de una misma capa convolucional.

La LRN se inspira en los mecanismos biológicos de **inhibición lateral** observados en el sistema visual humano, particularmente en la retina. En este proceso biológico, la activación de una célula nerviosa inhibe la respuesta de las neuronas vecinas, lo que incrementa el contraste y mejora la percepción de bordes y detalles. De manera análoga, la LRN permite que una neurona con una activación alta **reduzca la magnitud de las activaciones de las neuronas cercanas**, resaltando así aquellas respuestas más relevantes y disminuyendo la redundancia entre filtros.

El procedimiento de la LRN puede describirse del siguiente modo: para cada neurona activada, se considera un conjunto reducido de canales adyacentes (por ejemplo, los cinco canales circundantes). La activación de la neurona se **normaliza dividiéndola por un factor dependiente de la energía local**, es decir, de la suma de los cuadrados de las activaciones dentro de esa vecindad. En consecuencia, las neuronas con activaciones significativamente superiores a las de sus vecinas mantienen su valor elevado, mientras que aquellas con activaciones más bajas son atenuadas. Esta dinámica fomenta la especialización de los filtros y contribuye a una representación más discriminativa de las características.

A pesar de su utilidad inicial, la LRN fue gradualmente **reemplazada por métodos de normalización más eficientes y estables**, tales como **Batch Normalization (BN)**, **Layer Normalization (LN)** e **Instance Normalization (IN)**. Estas técnicas ofrecen **mayor estabilidad numérica**, **aceleran el entrenamiento** y **mejoran el rendimiento general** de las redes profundas. En la práctica moderna, el uso de LRN es escaso, dado que las nuevas estrategias de normalización resultan **más simples, robustas y efectivas** en una amplia variedad de arquitecturas y contextos de aprendizaje profundo.

Paper: https://www.cs.toronto.edu/~fritz/absps/imagenet.pdf

In [1]:
# 3pps
import torch
from torch import nn

In [2]:
# 3pps
import torch
import torch.nn as nn


class LocalResponseNormalization(nn.Module):

    def __init__(
        self, k: float = 2.0, n: int = 5, alpha: float = 1e-4, beta: float = 0.75
    ) -> None:
        super().__init__()
        self.k = k
        self.n = n
        self.alpha = alpha
        self.beta = beta

    def forward(self, input_tensor: torch.Tensor) -> torch.Tensor:
        batch, channels, height, width = input_tensor.shape
        response_normalization = input_tensor.clone()

        for channel in range(channels):
            for x in range(height):
                for y in range(width):
                    end_iterator = min(channels - 1, (channel + self.n) // 2)
                    start_iterator = max(0, (channel - self.n) // 2)
                    numerator = input_tensor[:, channel, x, y]
                    denominator = (
                        self.k
                        + self.alpha
                        * sum(
                            (input_tensor[:, i, x, y] ** 2)
                            for i in range(start_iterator, end_iterator + 1)
                        )
                    ) ** self.beta

                    response_normalization[:, channel, x, y] = numerator / denominator

        return response_normalization

In [3]:
lrn = LocalResponseNormalization()
x = torch.randn(1, 10, 32, 32)
y = lrn(x)

### Global Response Normalization

La **Global Response Normalization (GRN)** es una técnica reciente en el ámbito de la visión por computadora, introducida en el trabajo [*“ConvNeXt V2: Co-designing and Scaling ConvNets with Masked Autoencoders”*](https://arxiv.org/pdf/2301.00808).

Esta técnica se incorpora como una capa de normalización global cuyo propósito principal es fomentar la competencia entre canales dentro de los mapas de características de las redes convolucionales. Su implementación busca mitigar el fenómeno conocido como *feature collapse*, frecuente en autoencoders enmascarados completamente convolucionales. El *feature collapse* ocurre cuando varios canales de una red neuronal presentan redundancia o pérdida de diversidad. En tales casos, algunos canales pueden generar activaciones constantes o saturarse, reduciendo la variabilidad de las representaciones internas y, en consecuencia, la calidad de las características aprendidas. GRN aborda este problema mediante un proceso de normalización y recalibración que equilibra las contribuciones de los distintos canales.

El mecanismo de GRN se estructura en tres etapas fundamentales. Considerando un tensor de activaciones **X** con dimensiones **(N, C, H, W)**, correspondientes a tamaño de lote, número de canales, altura y anchura, el proceso se desarrolla de la siguiente manera:

1. **Agregación global de características:**
   Para cada canal *i*, se calcula una norma global (usualmente la norma $L_2$) a partir de todos los valores espaciales del mapa de características.
   $$
   G_i = \sqrt{\sum_{h,w} X_{i,h,w}^2}
   $$
   Este cálculo produce un vector **$G(X) = [G₁, G₂, …, G_C]$**, que representa la magnitud global de activación de cada canal.

2. **Normalización intercanal:**
   Posteriormente, los valores de norma se normalizan entre canales, dividiéndose cada uno por la media global de las normas o por otra estadística equivalente.
   $$
   N_i = \frac{G_i}{\mathrm{mean}(G(X)) + \epsilon}
   $$
   Este paso genera un factor de ponderación relativo que indica la intensidad de activación de cada canal respecto al resto.

3. **Recalibración de características y conexión residual:**
   Cada canal del mapa de entrada se reescala multiplicándolo por su correspondiente factor normalizado (**Nᵢ**), aplicando además un escalado (**γ**) y un sesgo (**β**) aprendibles, junto con una conexión residual hacia la entrada original:
   $$
   \text{Output}_i = \gamma \cdot (X_i \cdot N_i) + \beta + X_i
   $$
   Los parámetros **γ** y **β** se ajustan durante el entrenamiento y son específicos de cada canal.

In [4]:
class GlobalResponseNormalization(nn.Module):

    def __init__(self, num_channels: int, eps: float = 1e-6) -> None:

        super().__init__()

        self.num_channels = num_channels
        self.eps = eps

        self.gamma = nn.Parameter(data=torch.zeros(size=(1, self.num_channels, 1, 1)))
        self.beta = nn.Parameter(data=torch.zeros(size=(1, self.num_channels, 1, 1)))

    def forward(self, input_tensor: torch.Tensor) -> torch.Tensor:

        gx = torch.norm(input_tensor, p=2, dim=(2, 3), keepdim=True)
        nx = gx / (gx.mean(dim=1, keepdim=True) + self.eps)

        return self.gamma * (input_tensor * nx) + self.beta + input_tensor

In [5]:
x = torch.randn(8, 128, 32, 32)
grn = GlobalResponseNormalization(x.shape[1])
y = grn(x)
print(y.shape)

torch.Size([8, 128, 32, 32])


### Batch Normalization

In [6]:
class BatchNormalization2D(nn.Module):
    def __init__(self, num_channels: int, eps: float = 1e-6) -> None:

        super().__init__()

        self.num_channels = num_channels
        self.eps = eps

        # For inference
        self.register_buffer("running_mean", torch.zeros(1, num_channels, 1, 1))
        self.register_buffer("running_std", torch.ones(1, num_channels, 1, 1))

    def forward(self, input_tensor: torch.Tensor) -> torch.Tensor:

        # Input tensor -> (B, C, H, W) -> Batch norm is applied for each batch in H X W
        mean = x.mean(dim=(0, 2, 3), keepdim=True)
        std = x.std(dim=(0, 2, 3), keepdim=True)

        self.running_mean = mean.detach()
        self.running_std = std.detach()

        return (x - mean) / (std + self.eps)

In [7]:
x = torch.randn(8, 128, 32, 32)
bn = BatchNormalization2D(num_channels=x.shape[1]).train()
y = bn(x)
print(y.shape)

bn.eval()
with torch.no_grad():
    y = bn(x)
print(y.shape)

torch.Size([8, 128, 32, 32])
torch.Size([8, 128, 32, 32])


### Layer Normalization

In [8]:
class LayerNormalization2D(nn.Module):

    def __init__(self, num_channels: int, eps: float = 1e-6) -> None:

        super().__init__()

        self.num_channels = num_channels
        self.eps = eps

    def forward(self, input_tensor: torch.Tensor) -> torch.Tensor:

        mean = input_tensor.mean(dim=(1, 2, 3), keepdim=True)
        std = input_tensor.mean(dim=(1, 2, 3), keepdim=True)

        return (x - mean) / (std + self.eps)

In [9]:
x = torch.randn(8, 128, 32, 32)
ln = LayerNormalization2D(num_channels=x.shape[1]).train()
y = ln(x)
print(y.shape)

torch.Size([8, 128, 32, 32])
